# Survival Analysis TSR

In [ ]:
import os
import numpy as np
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
s

%matplotlib inline

In [ ]:
def softmax(x,axis=None):
    x_max = np.amax(x, axis=axis, keepdims=True)
    exp_x_shifted = np.exp(x - x_max)
    return exp_x_shifted / np.sum(exp_x_shifted, axis=axis, keepdims=True)
    
def display_auc(X1,X10,X100,y1,y10,y100,name=''):
    cv = KFold(n_splits=5, shuffle=False)

    tprs_old = []
    aucs_old = []
    old_predictions, new_predictions,avg_predictions = [],[],[]
    old_labels,new_labels,avg_labels = [],[],[]
    tprs_new = []
    aucs_new = []
    
    tprs_avg = []
    aucs_avg = []
    mean_fpr = np.linspace(0, 1, 100)

    fig, ax = plt.subplots(figsize=(15,10))

    for i, (train, test) in enumerate(cv.split(X1, y1)):
        classifier_old = LogisticRegression()
        classifier_old.fit(X1[train], y1[train])
        y_pred = classifier_old.decision_function(X1[test])
        my_prediction = classifier_old.predict_proba(X1[test])
        my_prediction = softmax(my_prediction, axis=1)[:,1]
        old_predictions.append(my_prediction)
        old_labels.append(y1[test])
        fpr, tpr, thresholds = metrics.roc_curve(y1[test],y_pred)
        roc_auc = metrics.auc(fpr, tpr)
        viz = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                      estimator_name="ROC fold {}".format(i))

        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs_old.append(interp_tpr)
        aucs_old.append(viz.roc_auc)


    for i, (train, test) in enumerate(cv.split(X10, y10)):
        classifier_new = LogisticRegression()
        classifier_new.fit(X10[train], y10[train])
        y_pred = classifier_new.decision_function(X10[test])
        my_prediction = classifier_new.predict_proba(X10[test])
        my_prediction = softmax(my_prediction, axis=1)[:,1]
        new_predictions.append(my_prediction)
        new_labels.append(y10[test])
        fpr, tpr, thresholds = metrics.roc_curve(y10[test],y_pred)
        roc_auc = metrics.auc(fpr, tpr)
        viz = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                      estimator_name="ROC fold {}".format(i))

        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs_new.append(interp_tpr)
        aucs_new.append(viz.roc_auc)
    
    for i, (train, test) in enumerate(cv.split(X100, y100)):
        classifier_avg = LogisticRegression()
        classifier_avg.fit(X100[train], y100[train])
        y_pred = classifier_avg.decision_function(X100[test])
        my_prediction = classifier_avg.predict_proba(X100[test])
        my_prediction = softmax(my_prediction, axis=1)[:,1]
        avg_labels.append(y100[test])
        avg_predictions.append(my_prediction)
        fpr, tpr, thresholds = metrics.roc_curve(y100[test],y_pred)
        roc_auc = metrics.auc(fpr, tpr)
        viz = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                      estimator_name="ROC fold {}".format(i))

        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs_avg.append(interp_tpr)
        aucs_avg.append(viz.roc_auc)

    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

    mean_tpr_old = np.mean(tprs_old, axis=0)
    mean_tpr_old[-1] = 1.0
    mean_auc_old = auc(mean_fpr, mean_tpr_old)
    std_auc_old = np.std(aucs_old)
    ax.plot(
        mean_fpr,
        mean_tpr_old,
        color="b",
        label=r"TSR 6 months = %0.2f $\pm$ %0.2f" % (mean_auc_old, std_auc_old),
        lw=2,
        alpha=0.8,
    )

    mean_tpr_new = np.mean(tprs_new, axis=0)
    mean_tpr_new[-1] = 1.0
    mean_auc_new = auc(mean_fpr, mean_tpr_new)
    std_auc_new = np.std(aucs_new)
    ax.plot(
        mean_fpr,
        mean_tpr_new,
        color="r",
        label=r"TSR 12 months = %0.2f $\pm$ %0.2f" % (mean_auc_new, std_auc_new),
        lw=2,
        alpha=0.8,
    )
    mean_tpr_avg = np.mean(tprs_avg, axis=0)
    mean_tpr_avg[-1] = 1.0
    mean_auc_avg = auc(mean_fpr, mean_tpr_avg)
    std_auc_avg = np.std(aucs_new)
    ax.plot(
        mean_fpr,
        mean_tpr_avg,
        color="y",
        label=r"TSR 18 months = %0.2f $\pm$ %0.2f" % (mean_auc_avg, std_auc_avg),
        lw=2,
        alpha=0.8,
    )

    std_tpr_old = np.std(tprs_old, axis=0)
    tprs_upper_old = np.minimum(mean_tpr_old + std_tpr_old, 1)
    tprs_lower_old = np.maximum(mean_tpr_old - std_tpr_old, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower_old,
        tprs_upper_old,
        color="blue",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    std_tpr_new = np.std(tprs_new, axis=0)
    tprs_upper_new = np.minimum(mean_tpr_new + std_tpr_new, 1)
    tprs_lower_new = np.maximum(mean_tpr_new - std_tpr_new, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower_new,
        tprs_upper_new,
        color="red",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )
    
    std_tpr_avg = np.std(tprs_avg, axis=0)
    tprs_upper_avg = np.minimum(mean_tpr_avg + std_tpr_avg, 1)
    tprs_lower_avg = np.maximum(mean_tpr_avg - std_tpr_avg, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower_avg,
        tprs_upper_avg,
        color="yellow",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="Receiver operating characteristic",
    )
    ax.legend(loc="lower right")
#     plt.savefig('/data/pathology/projects/Pdac-epithelium-segmentation/Survival/Survival_all/shuffled_combined_'+name+'.png')
    plt.show()
    return np.concatenate(old_predictions),np.concatenate(old_labels),np.concatenate(new_predictions),np.concatenate(new_labels),np.concatenate(avg_predictions),np.concatenate(avg_labels)
    

## Prepare paths and dataframes

In [ ]:
clinical_path = '/data/pathology/projects/Pdac-Segmentation/clinical.cart.2021-12-13/Clean_clinical.tsv'
tsr_path = "/data/pathology/projects/Pdac-epithelium-segmentation/TSR_Final/Tsr_combined_TCGA.csv"

In [ ]:
tsv_data = pd.read_csv(clinical_path, sep='\t')
df1 = tsv_data[['Patient_id', 'Survival label', 'days_to_survival','gender', 'age_at_index', 'primary_diagnosis','prior_malignancy','origin']]
df1 = df1.drop_duplicates(subset='Patient_id', keep='last')
tsr_data_old = pd.read_csv(tsr_path, sep=',')
tsr_old_names = tsr_data_old['case_id']
tsr_old_names = [i[:12] for i in tsr_old_names]
tsr_old_names = np.unique(tsr_old_names)

assert len(tsr_old_names) == len(np.unique(tsr_old_names)), print('There are {} duplicates'.format(len(tsr_old_names)-len(np.unique(tsr_old_names))))
# df2 = tsr_data_old[['case_id', 'tsr_ratio', 'median_tsr', 'average_tsr']]
df2 = tsr_data_old[['case_id','tsr_ratio_CH']]
df2=df2.rename(columns={"tsr_ratio_CH": "tsr_ratio"})

df2['case_id'] = df2['case_id'].apply(lambda x: x[:12])


df_names = pd.DataFrame(tsr_old_names, columns =['case_id'])
# df_names = pd.DataFrame(tsr_old_names, columns =['case_id'])
df2 =pd.merge(df_names,df2, on='case_id' )
df2 = df2.sort_values(by='tsr_ratio')
df2 = df2.drop_duplicates('case_id', keep='first')

In [ ]:
df1=df1.rename(columns={"Patient_id": "case_id"})
df1=df1.rename(columns={"Survival label": "vital_status"})
df3 = df1.drop_duplicates(subset='case_id', keep='last')
my_df1 = pd.merge(df3, df2, on='case_id')
my_df1.loc[my_df1['vital_status']=='Alive', 'vital_status'] = 0
my_df1.loc[my_df1['vital_status']=='Dead', 'vital_status'] = 1
my_df1['age_at_index'] = my_df1['age_at_index'].apply(lambda x: x/100)
my_df1.loc[my_df1['gender']=='male', 'gender'] = 1
my_df1.loc[my_df1['gender']=='female', 'gender'] = 0

my_df1.loc[my_df1['prior_malignancy']=='no', 'prior_malignancy']                = 0
my_df1.loc[my_df1['prior_malignancy']=='yes', 'prior_malignancy']               = 1

In [ ]:
my_df_categorical_old = my_df1.copy()

In [ ]:
primary_diagnosis = pd.get_dummies(my_df_categorical_old['primary_diagnosis'], drop_first=True)
tissue_or_organ_of_origin = pd.get_dummies(my_df_categorical_old['origin'], drop_first = True)
my_df_categorical_old.drop(['primary_diagnosis', 'origin'], axis=1, inplace=True)
my_df_categorical_old = pd.concat([my_df_categorical_old, primary_diagnosis, tissue_or_organ_of_origin], axis=1)

In [ ]:
my_df_categorical_old.head()

## Logistic Regression

In [ ]:
threshold  = 180
threshold_2 = 365
threshold_3 = 540

my_df_categorical_old['state_6'] = my_df_categorical_old.apply(lambda row: 1 if (row['vital_status'] == 1) and (row['days_to_survival'] <= threshold) else 0, axis=1)
my_df_categorical_old['state_12'] = my_df_categorical_old.apply(lambda row: 1 if (row['vital_status'] == 1) and (row['days_to_survival'] <= threshold_2) else 0, axis=1)
my_df_categorical_old['state_18'] = my_df_categorical_old.apply(lambda row: 1 if (row['vital_status'] == 1) and (row['days_to_survival'] <= threshold_3) else 0, axis=1)

In [ ]:
my_df_categorical_old.head()

In [ ]:
df_categorical_shuffled = my_df_categorical_old.sample(frac=1, random_state=42)
df_categorical_shuffled.head()

In [ ]:
feats = ['gender', 'age_at_index', 'prior_malignancy', 'tsr_ratio', 'Infiltrating duct carcinoma, NOS', 'Pancreas, NOS', 'Tail of pancreas']

Xc1 = df_categorical_shuffled[feats]
yc1 = df_categorical_shuffled['state_6']
Xc1 = np.asarray(Xc1)
yc1 = np.asarray(yc1).astype('int')


yc10 = df_categorical_shuffled['state_12']
yc10 = np.asarray(yc10).astype('int')


yc100 = df_categorical_shuffled['state_18']
yc100 = np.asarray(yc100).astype('int')


short_pred,short_labels,one_y_pred,one_y_labels,eighteen_m_pred,eightee_m_labels =display_auc(Xc1,Xc1,Xc1,yc1,yc10,yc100,name='Features')